<a href="https://colab.research.google.com/github/Xessen/HotDogvNotHotDog/blob/main/DeepLearning_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [164]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
import matplotlib.image as mpimg
import cv2
import os


In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")
!cp -r '/content/gdrive/MyDrive/n_hotdog' n_hotdog
!cp -r '/content/gdrive/MyDrive/hotdog' hotdog

Mounted at /content/gdrive


In [ ]:
len(os.listdir("/content/hotdog"))

499

In [170]:
x_test=[]
y_test=[[]]
x_train=[]
y_train=[[]]
import random
path=["/content/hotdog","/content/n_hotdog"]
def preprocess_image(pathlist):
  global x_test,x_train,y_test,y_train
  for path in pathlist:
    numberoffiles=len(os.listdir(path))
    counter=0
    for img in os.listdir(path):
      #Yüzde 70'ini training 30'unu test olarak ayırıyor.
      if counter<numberoffiles*0.7:
        #cv2'nin sıradan renk şeması bgr onu rgb'ye çevirip rgb kodlarını pic'e atıyor
        pic = cv2.cvtColor(cv2.imread(os.path.join(path,img)),cv2.COLOR_BGR2RGB)
        #tekrardan okuyup bgr halinden çıkmasını sağlıyor
        pic = cv2.imread(os.path.join(path,img))
        #resizelıyor
        pic = cv2.resize(pic,(80,80))
        
        x_train.append([pic])
        #Manuel ayarlanması gereken bir kısım hangi fotoğrafların 1 hangilerinin 0 değeri alacağını belirliyor
        if path=="/content/hotdog":
          y_train[0].append(1)
        else:
          y_train[0].append(0)
        counter+=1
      else:
        pic = cv2.cvtColor(cv2.imread(os.path.join(path,img)),cv2.COLOR_BGR2RGB)
        pic = cv2.imread(os.path.join(path,img))
        pic = cv2.resize(pic,(80,80))
        x_test.append([pic])
        if path=="/content/hotdog":
          y_test[0].append(1)
        else:
          y_test[0].append(0)

        counter+=1
  #eğer accuracy absürd derecede düşük çıkarsa bu kısmı sil at demekki sequence karışmış
  temp0=list(zip(x_train,y_train[0]))
  random.shuffle(temp0)
  x_train,y_train[0]=zip(*temp0)
  temp1=list(zip(x_test,y_test[0]))
  random.shuffle(temp1)
  x_test,y_test[0]=zip(*temp1)
  
  #listeleri numpy arraylere çeviriyor.
  x_train=np.array(x_train)
  y_train=np.array(y_train)
  x_test=np.array(x_test)
  y_test=np.array(y_test)

  #Fotoğraf sayısı kadar sütun,piksel sayısı*3(r-g-b) satır olan matrixe dönüşütülürüz(flattening)
  x_train=x_train.reshape(x_train.shape[0],-1).T
  x_test=x_test.reshape(x_test.shape[0],-1).T
  #işlem kolaylığı sağlansın diye 255'e oranlanır
  x_train=x_train/255.
  x_test=x_test/255.

preprocess_image(path)

In [171]:
print(x_test.shape)

(19200, 298)


In [172]:
def sigmoid(x):
  s=1/(1+np.exp(-x))
  return s

In [173]:
def propagate(w,b,X,Y):
  m=X.shape[1]
  #ileri propagasyon
  A=sigmoid(np.dot(w.T,X)+b)
  J=(-1/m)*np.sum(Y*np.log(A)+(1-Y)*np.log(1-A))

  #geri propagasyon
  dw=(1/m)*np.dot(X,(A-Y).T)
  db=(1/m)*np.sum(A-Y)

  J=np.squeeze(J)

  grads={
      'dw':dw,
      'db':db
  }
  return grads,J

In [174]:
def optimization(w,b,X,Y,num_iter,learning_rate):
  for i in range(num_iter):
    #num_iter kez propagate yapılır her seferinde w ve b'nin türevi alındığınından değerler hata değerini minimuma götürür
    grads,J=propagate(w,b,X,Y)
    w=w-learning_rate*grads['dw']
    b=b-learning_rate*grads['db']
  parameter={
      'w':w,
      'b':b
  }
  return parameter,grads,J

In [175]:
parameter,grads,J=optimization(np.array([[1.],[2.]]), 2., np.array([[1.,2.,-1.],[3.,4.,-3.2]]), np.array([[1,0,1]]),100,0.09)
print(parameter['w'])

[[-0.64078657]
 [-0.43895744]]


In [176]:
def predict(w,b,X):
  m = X.shape[1]
  prediction = np.zeros((1,m))
  #w'yu (m,1) şekline sokarız ki hiç bir X değeri boşta kalmasın ve her fotoğraftaki X değerleri için broadcasting yapsın
  w=w.reshape(X.shape[0],1)
  A=sigmoid(np.dot(w.T,X)+b)
  for i in range(A.shape[1]):
    prediction[0,i]=round(A[0,i])
  return prediction

In [177]:
def w_b(x):
  #w değerlerini başlatmak için x boyutlu 0'lar kümesi oluşturuyor
  w=np.zeros(x)
  b=0
  return w,b

In [198]:
def nn(x_train,x_test,y_train,y_test,num_iter,learning_rate):
  w,b=w_b((x_train.shape[0],1))
  parameter,grads,J=optimization(w,b,x_train,y_train,num_iter,learning_rate)
  w=parameter['w']
  b=parameter['b']
  prediction_train=predict(w,b,x_train)
  prediction_test=predict(w,b,x_test)
  print(f"Training accuracy: {100-np.mean(np.abs(prediction_train-y_train))*100}")
  print(f"Testing accuracy: {100-np.mean(np.abs(prediction_test-y_test))*100}")
  return w,b


In [204]:
w,b=nn(x_train,x_test,y_train,y_test,300,0.0041)

(19200, 700)
(19200, 1)
(19200, 1)
(19200, 298)
(19200, 1)
(19200, 1)
Training accuracy: 55.0
Testing accuracy: 53.691275167785236


In [202]:
print(b)

-1.8417565561122606
